In [ ]:
from MA4Strategy import generate_candlestick_with_emas, load_data, evaluate_model
from MA4Strategy import build_cnn_model, build_cnn_lstm_model, build_dense_model, plot_accuracy, cal_accuracy
import pandas as pd
import tensorflow as tf
import os
import pickle

In [ ]:
result_file = "4MAs_result.csv"

In [ ]:
def main(ticker, window_days, alpha, epochs, predict_days, batch_size, result_df):
    # if model file exist, skip the process
    model_file = os.path.join("models", f"{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}.pkl")
    if os.path.exists(model_file):
        print(f"{model_file} model exist")
        return result_df

    # Generate and split data
    train_p = f"train_{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}"
    test_p = f"test_{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}"
    train_charts, train_labels, test_charts, test_labels, fulldata = generate_candlestick_with_emas(ticker, predict_days, window_days, alpha,
                                                                                                   train_p, test_p)

    if not train_charts or not test_charts:
        print("No charts generated, exiting.")
        return result_df
        
    em_y = fulldata['EMStrend'].dropna()
    print(len(em_y))
    lb_y = fulldata['Labels'].dropna()
    print(len(lb_y))

    # Load training and testing data
    X_train, y_train = load_data(f"{train_p}_metadata.csv")
    X_test, y_test = load_data(f"{test_p}_metadata.csv")
    
    # print("Trainning data:  ", X_train, ",", y_train)
    # print("Testing data:  ", X_test, ",", y_test)
    
    if X_train.size == 0 or X_test.size == 0:
        print("No valid images processed, exiting.")
        return result_df
        
    # Define models to train
    models = [build_cnn_model(), build_cnn_lstm_model(), build_dense_model()]
    
    # Train and evaluate each model
    for model, model_name in models:
        print(f"\nTraining {model_name}...")
        history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, batch_size=batch_size, verbose=1)
        plot_accuracy(ticker, history, model_name)
        acc, prec, recall, f1 = evaluate_model(ticker, model, X_test, y_test, model_name)
        # Append the row
        new_row =[ticker,model_name,window_days, alpha, epochs, predict_days, batch_size,acc, prec, recall, f1]
        result_df.loc[len(result_df)] = new_row
        with open(model_file, 'wb') as file:
            pickle.dump(model, file)   
        result_df.to_csv(result_file, index=False)
        tf.keras.backend.clear_session()

    trend_mapping = {"UP": 1, "DOWN": 2, "SIDEWAY": 3}
    em_y = fulldata['EMStrend'].dropna().to_list()
    lb_y = fulldata['Labels'].dropna().to_list()
    emyy = [trend_mapping[em] for em in em_y]
    lbyy = [trend_mapping[em] for em in lb_y]

    acc, prec, recall, f1 = cal_accuracy(ticker, lbyy, emyy, model_name)
    new_row =[ticker,'4MAs',window_days, alpha, epochs, predict_days, batch_size,acc, prec, recall, f1]
    result_df.loc[len(result_df)] = new_row
    result_df.to_csv(result_file, index=False)
    return result_df

In [ ]:
tickers = ['^HSI','SPY','NVDA','TSLA']
window_days_l = [25, 50, 75]
alpha_l = [0.4, 0.5, 0.6]
epochs_l = [25,50]
predict_days_l = [5,8]
batch_size = 32

In [ ]:
column_names=['ticker','model','window_days','alpha','epochs','predict_days','batch_size','accuracy','precision','recall','f1']

if os.path.exists(result_file):
    print(f"Reload {result_file} for append result.")
    result = pd.read_csv(result_file)
else:
    print(f"Create {result_file}")
    result=pd.DataFrame(columns=column_names)

for ticker in tickers:
    for window_days in window_days_l:
        for alpha in alpha_l:
            for epochs in epochs_l:
                for predict_days in predict_days_l:
                    print( window_days, alpha, epochs, predict_days)
                    result = main(ticker, window_days, alpha, epochs, predict_days, batch_size, result)

In [ ]:
display(result)